In [ ]:
! pip install rasterio
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import rasterio
import numpy as np

In [ ]:
# Define the DiceLoss class
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        intersection = torch.sum(inputs * targets)
        cardinality = torch.sum(inputs) + torch.sum(targets)
        dice = (2. * intersection + self.smooth) / (cardinality + self.smooth)
        return 1 - dice

        # Instantiate the DiceLoss
dice_loss = DiceLoss()

# Instantiate the CrossEntropyLoss
cross_entropy_loss = nn.CrossEntropyLoss()
# Combining the two losses
def combined_loss(outputs, targets):
    ce_loss = cross_entropy_loss(outputs, targets)
    dice_loss_val = dice_loss(outputs, targets)
    return ce_loss + dice_loss_val


In [ ]:
def double_conv_3d(in_channels, out_channels):
    conv = nn.Sequential(
        nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
    )
    return conv

def crop_img_3d(tensor, target_sensor):
    target_size = target_sensor.size()[2]
    tensor_size = tensor.size()[2]
    delta = tensor_size - target_size
    delta = delta // 2
    return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta, delta:tensor_size-delta]

class UNet3D(nn.Module):
    def __init__(self):
        super(UNet3D, self).__init__()

        self.max_pool_2x2x2 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.down_conv_1 = double_conv_3d(1, 64)
        self.down_conv_2 = double_conv_3d(64, 128)
        self.down_conv_3 = double_conv_3d(128, 256)
        self.down_conv_4 = double_conv_3d(256, 512)
        self.down_conv_5 = double_conv_3d(512, 1024)

        self.up_trans_1 = nn.ConvTranspose3d(in_channels=1024, out_channels=512,
                                             kernel_size=2, stride=2)
        self.up_conv_1 = double_conv_3d(1024, 512)

        self.up_trans_2 = nn.ConvTranspose3d(in_channels=512, out_channels=256,
                                             kernel_size=2, stride=2)
        self.up_conv_2 = double_conv_3d(512, 256)

        self.up_trans_3 = nn.ConvTranspose3d(in_channels=256, out_channels=128,
                                             kernel_size=2, stride=2)
        self.up_conv_3 = double_conv_3d(256, 128)

        self.up_trans_4 = nn.ConvTranspose3d(in_channels=128, out_channels=64,
                                             kernel_size=2, stride=2)
        self.up_conv_4 = double_conv_3d(128, 64)

        self.out = nn.Conv3d(in_channels=64, out_channels=2, kernel_size=1)

    def forward(self, image):
        # Encoder
        x1 = self.down_conv_1(image)
        x2 = self.max_pool_2x2x2(x1)
        x3 = self.down_conv_2(x2)
        x4 = self.max_pool_2x2x2(x3)
        x5 = self.down_conv_3(x4)
        x6 = self.max_pool_2x2x2(x5)
        x7 = self.down_conv_4(x6)
        x8 = self.max_pool_2x2x2(x7)
        x9 = self.down_conv_5(x8)
        print("Encoded feature size:", x9.size())

        # Decoder
        x = self.up_trans_1(x9)
        y = crop_img_3d(x7, x)
        x = self.up_conv_1(torch.cat([x, y], 1))

        x = self.up_trans_2(x)
        y = crop_img_3d(x5, x)
        x = self.up_conv_2(torch.cat([x, y], 1))

        x = self.up_trans_3(x)
        y = crop_img_3d(x3, x)
        x = self.up_conv_3(torch.cat([x, y], 1))

        x = self.up_trans_4(x)
        y = crop_img_3d(x1, x)
        x = self.up_conv_4(torch.cat([x, y], 1))

        x = self.out(x)
        print(x.size())
        return x

# Example usage:
depth = 23
height = 846
width = 1262
model = UNet3D()
image = torch.rand(1, 1, depth, height, width)
output = model(image)
print(output.size())

In [ ]:


class TiffDataset(Dataset):
    def __init__(self, tif_path):
        self.tif_path = tif_path
        # Open the TIFF file
        with rasterio.open(tif_path) as src:
            # Read the image bands
            self.image = src.read()  # Shape: (bands, height, width)
            # Normalize the image bands
            self.image = self.image.astype(np.float32) / 255.0  # Assuming pixel values are in [0, 255]
            # Transpose the image to (height, width, bands) for PyTorch
            self.image = np.transpose(self.image, (1, 2, 0))

    def __len__(self):
        return 1  # Since we have only one image

    def __getitem__(self, idx):
        return torch.from_numpy(self.image.transpose((2, 0, 1))).unsqueeze(0)  # Add batch dimension

# Define your TIFF file path
tif_path = "/content/drive/MyDrive/Sentinal.tif"
# Create dataset and data loader
dataset = TiffDataset(tif_path)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)  # Assuming batch size of 1

# Iterate over the data loader
for images in data_loader:
    # images will contain the input image tensor
    print(images.shape)  # Shape: (batch_size, channels, height, width)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    # Iterate over the data loader
    for images in data_loader:
        # Transfer data to GPU if available
        images = images.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute the loss
        loss = combined_loss(outputs, targets)  # Assuming targets are available

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

    # Print the average loss for this epoch
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")
